In [ ]:
import pandas as pd
import numpy as np
import selenium3 as s
from pandas import Timestamp
import json
import redis
import time
import datetime
import ast
r = s.redis_conn(1)
r15 = s.redis_conn(15)

<br>

##### Method 1

In [ ]:
%%time
data1 = r.lrange('Ticks-FULL',0,0)
df = pd.DataFrame(eval(data1[0]))

In [ ]:
df

<br>

<br>

##### Method 2 - Make it in Dict

In [ ]:
%%time
data2 = r.lrange('Ticks-FULL',0,0)
string = data2[0]
d = eval(string.strip('[]'))
print(type(d))

In [ ]:
d

<br>

<br>

##### Method 3  Make into Dict to df

In [ ]:
%%time
data3 = r.lrange('Ticks-FULL',0,0)
df3 = pd.DataFrame.from_dict(eval(data3[0].strip('[]')),"index")
df3 = df3.transpose()

In [ ]:
df3

<br>

<br>

##### Method 4

In [ ]:
%%time
data4 = r.lrange('Ticks-FULL',0,0)
df4 = pd.DataFrame(eval(data4[0]))
df4.to_json()

In [ ]:
df4

<br>

##### Method 5 

In [ ]:
%%time
data5= r.lrange('Ticks-FULL',0,-1)
dict5_1 = eval(data5[0])
dict5_1_0 = dict5_1[0]
df5 = pd.DataFrame.from_dict([dict5_1_0],orient='columns')

In [ ]:
df5

<br>

##### Method 6 Process ticks in FULL mode

In [ ]:
%%time
my_dict = {}

data5= r.lrange('Ticks-FULL',0,-1)
for i in range(r.llen('Ticks-FULL')):
    dict5_1 = eval(data5[i])
    dict5_1_0 = dict5_1[0]
    my_dict[i] = dict5_1_0
df5 = pd.DataFrame.from_dict(my_dict,'index')

In [ ]:
df5[['instrument_token', 'change','last_price','last_trade_time','timestamp','volume']]

<br>

##### Method 7 Process Historical Data from Redis

In [ ]:
%%time
data7= r15.lrange('SBIN18MAYFUT',0,-1)
str7={}
my_dict7={}
for i in range(r15.llen('SBIN18MAYFUT')):    
    str7 = eval(data7[i])    
    str7['Stock'] = 'SBIN18MAYFUT'

    my_dict7[i] = str7
df7 = pd.DataFrame.from_dict(my_dict7,'index')   
df7['Millions'] = round(df7.volume/1000000,2)

In [ ]:
df7

In [ ]:
stocklist = r15.keys()

In [ ]:
%%time
j=0
stock_string = {}
stock_dict ={}    

for stock_name in stocklist:
    stock_data = r15.lrange(stock_name,0,-1)  
    for i in range(r15.llen(stock_name)):
        stock_string = eval(stock_data[i])
        stock_string['stock'] = stock_name
        stock_dict[j] = stock_string
        j+=1
    #break
df = pd.DataFrame.from_dict(stock_dict,'index')         

##### Get the Max volume of all the stocks

In [117]:
df.loc[df['volume'].idxmax()]

date       2018-05-23 09:17:00
open                       264
high                     266.2
low                        264
close                   265.65
volume                 7923000
Candle                   Green
TVolume               21813000
stock             SBIN18MAYFUT
Name: 42741, dtype: object

##### Get the Min volume of all the stocks

In [112]:
df.loc[df['volume'].idxmin()]

date       2018-05-23 12:34:00
open                      3508
high                      3508
low                       3508
close                     3508
volume                       0
Candle                   Green
TVolume                 715250
stock              TCS18MAYFUT
Name: 199, dtype: object

<br>